# Entropy estimation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
from scipy.optimize import fsolve

In [2]:
lag = 16
stats_count = lag + 4
output_dim = 1
input_data_dim = 20
training_set_ratio = 0.8
#batch_size

## Data-Preprocessing

In [3]:
import math

training_set_ratio = 0.8

with open('random_files/dev-random.bin','rb') as fr:
    fr.seek(0,2)
    file_size = fr.tell()
    training_set_size = math.floor(0.8*file_size)
    
    
    fr.seek(0,0)
    data_train = fr.read(training_set_size)
    data_test = fr.read()

In [4]:
byte_string = "{0:08b}".format(int.from_bytes(data_train,'big'))

In [5]:
X_data_train = [float(char) for char in byte_string[:-((training_set_size*8)%(input_data_dim+1))]]

In [6]:
Y_train = X_data_train[20::21]

In [7]:
del X_data_train[20::21]

In [8]:
X_data_train = torch.tensor(X_data_train)

In [9]:
X_data_train

tensor([1., 0., 0.,  ..., 0., 1., 1.])

In [10]:
X_data_train = X_data_train.view(((training_set_size*8)//(input_data_dim+1)), input_data_dim)

In [11]:
X_data_train

tensor([[1., 0., 0.,  ..., 1., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 1., 1.],
        [1., 0., 1.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])

In [12]:
Y_train = torch.tensor(Y_train)

In [13]:
Y_train = Y_train.view(((training_set_size*8)//(input_data_dim+1)), 1)

In [14]:
Y_train

tensor([[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]])

In [15]:
byte_string = "{0:08b}".format(int.from_bytes(data_test,'big'))

In [16]:
len(byte_string)

203167

In [17]:
X_data_test = [float(char) for char in byte_string[:-((((file_size-training_set_size)*8)%(input_data_dim+1))-1)]]

In [18]:
len(X_data_test)

203154

In [19]:
Y_test = X_data_test[20::21]

In [20]:
del X_data_test[20::21]
X_data_test = torch.tensor(X_data_test)
X_data_test = X_data_test.view((((file_size-training_set_size)*8)//(input_data_dim+1)), input_data_dim)
Y_test = torch.tensor(Y_test)
Y_test = Y_test.view((((file_size-training_set_size)*8)//(input_data_dim+1)), 1)

## Neural Network

In [21]:
class Neural_Network(nn.Module):
    def __init__(self, stats_count, input_data_dim, num_classes):
        super(Neural_Network, self).__init__()
        self.fc1 = nn.Linear(stats_count, 20)
        self.fc2 = nn.Linear(20, 1)
        self.fc3 = nn.Linear(input_data_dim+1, 128)
        self.fc4 = nn.Linear(128, 20)
        self.fc5 = nn.Linear(20, num_classes)
    
    
    def forward(self, stats, input_data):
        x = F.relu(self.fc1(stats))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(torch.cat((input_data, x), dim=1)))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))        #softmax
        return x

model = Neural_Network(stats_count, input_data_dim, output_dim)
print(model)

Neural_Network(
  (fc1): Linear(in_features=20, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
  (fc3): Linear(in_features=21, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=20, bias=True)
  (fc5): Linear(in_features=20, out_features=1, bias=True)
)


In [22]:
loss_function = nn.L1Loss()
total_epochs = 20
learning_rate = 0.001    #update the equation
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
X_stats_train = torch.randn(((training_set_size*8)//(input_data_dim+1)), 20)

In [24]:
X_stats_train

tensor([[-0.3393,  0.7744,  0.6375,  ...,  0.2125, -0.5110,  1.1264],
        [-0.9365,  0.9669,  0.6278,  ...,  0.1219,  0.6159, -0.4203],
        [-0.1097, -1.7793, -0.7491,  ...,  0.3110, -0.1566,  0.3478],
        ...,
        [-0.2670, -1.0399,  0.3193,  ...,  0.6163,  1.5024,  0.9750],
        [-0.8288, -0.9271,  0.2054,  ...,  0.8639,  0.3082,  1.0368],
        [ 1.2804,  0.9247,  1.5558,  ...,  0.4599,  1.6363,  0.2233]])

In [25]:
output = model(X_stats_train,X_data_train)

In [26]:
for epoch in range(total_epochs):
    model.zero_grad()
    output = model(X_stats_train, X_data_train)
    #print(output)
    
    loss = loss_function(output, Y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(loss)
    

tensor(0.4969, grad_fn=<L1LossBackward>)
tensor(0.4968, grad_fn=<L1LossBackward>)
tensor(0.4967, grad_fn=<L1LossBackward>)
tensor(0.4967, grad_fn=<L1LossBackward>)
tensor(0.4967, grad_fn=<L1LossBackward>)
tensor(0.4967, grad_fn=<L1LossBackward>)
tensor(0.4967, grad_fn=<L1LossBackward>)
tensor(0.4967, grad_fn=<L1LossBackward>)
tensor(0.4966, grad_fn=<L1LossBackward>)
tensor(0.4966, grad_fn=<L1LossBackward>)
tensor(0.4965, grad_fn=<L1LossBackward>)
tensor(0.4965, grad_fn=<L1LossBackward>)
tensor(0.4964, grad_fn=<L1LossBackward>)
tensor(0.4964, grad_fn=<L1LossBackward>)
tensor(0.4963, grad_fn=<L1LossBackward>)
tensor(0.4962, grad_fn=<L1LossBackward>)
tensor(0.4961, grad_fn=<L1LossBackward>)
tensor(0.4960, grad_fn=<L1LossBackward>)
tensor(0.4959, grad_fn=<L1LossBackward>)
tensor(0.4957, grad_fn=<L1LossBackward>)


In [27]:
X_stats_test = torch.randn((((file_size-training_set_size)*8)//(input_data_dim+1)), 20)

In [28]:
with torch.no_grad():
    #for data in testset:
    output = model(X_stats_test, X_data_test)
    '''
    for idx, i in enumerate(output):
    if torch.argmax(i) == y[idx]:
    correct += 1
    total += 1
    '''
    loss = loss_function(output, Y_test)
    print(output)
    print(loss)

tensor([[0.0635],
        [0.1903],
        [0.0946],
        ...,
        [0.0627],
        [0.0537],
        [0.0786]])
tensor(0.5054)


## Entropy calculation

In [29]:
n = 70       #number of bits produced
c = 20       #number of correct bits
l = 5        #longest run

In [30]:
prediction_global = c/n

if prediction_global == 0 :
    prediction_global_normalized = 1 - (0.01**(1/n))

else:
    prediction_global_normalized = min(1, prediction_global+2.579*(((prediction_global*(1-prediction_global))/(n-1))**(1/2))) #99% confidence

In [31]:
def local_predictor(p, *args):
    
    l, n, precision = args
    
    
    q = 1-p
    x = 1+q*(p**l)
    
    
    for i in range(int(precision)):
        x = x + (((l+1)**(i+1))*((q*(p**l))**(i+2)))     #check equation
    
    return ((1-p*x)/((l+2-(l+1)*x)*q*(x**(n+1))))-0.99


In [32]:
predict =  fsolve(local_predictor, 0.5, (l, n, 0))

print(predict)
print(local_predictor(predict, l, n, 0))

[0.17979928]
[-7.32747196e-15]


In [33]:
precision = 0
efselon = 1e-5


predict =  fsolve(local_predictor, 0.5, (l, n, precision))
precision = precision+1

predict_new = fsolve(local_predictor, 0.5, (l, n, precision))


while abs(predict-predict_new)>efselon:
    precision = precision+1
    predict = predict_new
    predict_new = fsolve(local_predictor, 0.5, (l, n, precision))


prediction_local = predict_new

In [34]:
min_Entropy = -math.log(max(prediction_global_normalized, prediction_local),2)

In [35]:
min_Entropy

1.2311673053919407